In [1]:
import os
import numpy as np
import pandas as pd
import geopandas
import matplotlib

import holoviews as hv
import shapely

from bokeh.palettes import Viridis6 as palette
from bokeh.models import BoxZoomTool, LogColorMapper, LogTicker, ColorBar, LinearColorMapper, BasicTicker

from bokeh.plotting import figure, output_notebook, show
from taxiDataset import *
from torch.utils.data import Dataset, DataLoader
import bokeh
import torch

In [5]:
PU_dt = 'tpep_pickup_datetime'
PU_loc = 'PULocationID'
PU_dow = 'day_of_week'
bucket = 't_bucket'
dl = [PU_loc,PU_dow,bucket]

# taxi_dataset = taxiDataset(csv_file='yellow_tripdata_2016-12.csv', desired_labels=dl)
# df = taxi_dataset.df
# df_set = df.loc[(df[PU_dow] == 4)&(df[bucket]==36)].reset_index(drop=True)

df = pd.read_pickle('./assets/activity_plot.pickle')
print(df.head())
freq = df
# freq = df_set.drop([PU_dow,bucket],axis=1).reset_index(drop=True)

   LocationID    0
0           1   28
1           3    3
2           4  155
3           6    1
4           7  176


In [6]:
freq.columns = ['LocationID', 'freq']

# print(freq.keys())

In [7]:
# TODO: Fetch CSV with location ID codes.
taxi_id = geopandas.read_file(os.path.join("..", "assets", "taxi_zones", "taxi_zones_mercator.shp"))
print(taxi_id.head())

# taxi_id_shape.plot()

def getXYCoords(geometry, coord_type):
    """ Returns either x or y coordinates from  geometry coordinate sequence. Used with LineString and Polygon geometries."""
    if coord_type == 'x':
        return geometry.coords.xy[0]
    elif coord_type == 'y':
        return geometry.coords.xy[1]

def getPolyExtCoords(geometry, coord_type):
    """ Returns Coordinates of Polygon using the Exterior of the Polygon."""
    ext = geometry.exterior
    return getXYCoords(ext, coord_type)


def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    if type(row[geom]) != shapely.geometry.multipolygon.MultiPolygon:
        exterior = row[geom].exterior

        if coord_type == 'x':
            # Get the x coordinates of the exterior
            return list( exterior.coords.xy[0] )
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( exterior.coords.xy[1] )
    else:
        geoms_list = row[geom].geoms
        exterior_list = []
        
        geom_type = 'MultiPolygon'
        for i, part in enumerate(geoms_list):
            # On the first part of the Multi-geometry initialize the coord_array (np.array)
            if i == 0:
                coord_arrays = np.append(getPolyExtCoords(part, coord_type), np.nan)
            else:
                coord_arrays = np.concatenate([coord_arrays, np.append(getPolyExtCoords(part, coord_type), np.nan)])

        # Return the coordinates
        return coord_arrays

taxi_id['x'] = taxi_id.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
taxi_id['y'] = taxi_id.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)
# taxi_id['freq'] = np.random.rand(263,)
taxi_id = pd.merge(taxi_id, freq, on='LocationID')
# taxi_id.join(freq, on='LocationID')
# print(taxi_id.count)
# TODO: Find geofences for the given id codes.

# TODO: Overlay heatmap with geofences...
# taxi_id.plot()
taxi_id = taxi_id.drop('geometry', axis=1)

   OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   

         borough                                           geometry  
0            EWR  POLYGON ((-8258175.532737532 4967457.202992616...  
1         Queens  (POLYGON ((-8217980.621910957 4959237.28547167...  
2          Bronx  POLYGON ((-8220713.534155379 4993383.154018582...  
3      Manhattan  POLYGON ((-8234500.226961649 4971984.093397928...  
4  Staten Island  POLYGON ((-8257036.10884249 4948033.094989426,...  


In [8]:
taxi_id.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,x,y,freq
0,1,0.116357,0.000782,Newark Airport,1,EWR,"[-8258175.532737532, -8258179.540239202, -8258...","[4967457.202992616, 4967471.738424489, 4967484...",28
1,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"[-8220713.534155379, -8220638.427674196, -8220...","[4993383.154018582, 4993330.866827082, 4993311...",3
2,4,0.043567,0.000112,Alphabet City,4,Manhattan,"[-8234500.226961649, -8234502.155010963, -8234...","[4971984.093397928, 4971983.056324019, 4971983...",155
3,6,0.150491,0.000606,Arrochar/Fort Wadsworth,6,Staten Island,"[-8244730.385678907, -8244711.736101975, -8244...","[4953841.8729588045, 4953834.818856421, 495389...",1
4,7,0.107417,0.000390,Astoria,7,Queens,"[-8226970.8291156, -8226872.393520025, -822684...","[4978111.202464147, 4978108.851625502, 4978109...",176


In [9]:
output_notebook()

NYC = x_range, y_range = ((-8242000, -8210000), (4965000, 4990000))

plot_width  = int(750)
plot_height = int(plot_width//1.2)

def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    p.add_tools(BoxZoomTool(match_aspect=True))
    
    return p

options = dict(line_color=None, fill_color='blue', size=5)
color_mapper = LogColorMapper(palette="Inferno256", low=1, high=5000)


from bokeh.tile_providers import CARTODBPOSITRON_RETINA

p = base_plot()
p.add_tile(CARTODBPOSITRON_RETINA)
p.patches('x', 'y', source=taxi_id, 
          fill_color={'field': 'freq', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

p.add_layout(color_bar, 'right')

show(p)
# bokeh.io.export_png(p, filename='heat_map.png')

Loading BokehJS ...